In [1]:
from datasets import *

## 从 VisiumHD 生成 Visium
读取文件

In [2]:
path = '/data/datasets/Visium_HD_Human_Tonsil_Fresh_Frozen/binned_outputs/square_002um'
# path = '/mnt/TenTA-f702/user/zhangyichi/Data/Visium_HD_Human_Tonsil_Fresh_Frozen/binned_outputs/square_002um'
imagefile = '/data/datasets/Visium_HD_Human_Tonsil_Fresh_Frozen/Visium_HD_Human_Tonsil_Fresh_Frozen_tissue_image.tif'
# imagefile = '/mnt/TenTA-f702/user/zhangyichi/Data/Visium_HD_Human_Tonsil_Fresh_Frozen/Visium_HD_Human_Tonsil_Fresh_Frozen_tissue_image.tif'
rawdata = VisiumHDData()
rawdata.load(path=path, source_image_path=imagefile)

Start load
Finish load in 25.3900 s


In [3]:
visium_profile = VisiumProfile(slide_serial=4)
visium_profile.spot_step = 99.0

In [ ]:
emulate_visium = rawdata.HD2Visium(visium_profile)

In [8]:
save_path = Path('test/HD2Visium')
emulate_visium.save(save_path)

Start save
Finish save in 7.6002 s


## 使用 iStar 生成 Visium 的高分辨率图像 

In [2]:
visium_path = Path('test/HD2Visium')
iStar_visium = iStar()
iStar_visium.load(path=visium_path, source_image_path=visium_path/"image.tif")

In [3]:
mask = iStar_visium.tissue_mask(auto_mask=True)
iStar_visium.select_HVG(n_top_genes=2000)

In [4]:
iStar_visium.set_super_pixel_size(8)

In [3]:
istar_dir = Path("test/istar")
iStar_visium.save_inpout(istar_dir)

conda 环境切换到 iStar， 在 shell 下运行 iStar 工具

iStar 工具结束后读取结果

In [4]:
iStar_visium.load_output(istar_dir)

In [6]:
# iStar_visium.to_csv()
iStar_visium.to_h5ad()

Start to_h5ad


/home/yiriso/anaconda3/envs/DataReader/lib/python3.10/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


Finish to_h5ad in 0.8186 s


## 使用 iStar 从 Visium 生成 VisiumHD 数据

In [3]:
visium_path = Path('test/HD2Visium')
iStar_visium = iStar()
iStar_visium.load(path=visium_path, source_image_path=visium_path/"image.tif")
iStar_visium.tissue_mask(auto_mask=True)
iStar_visium.select_HVG(n_top_genes=2000)

设定 VisiumHD 的配置

In [9]:
visiumHD_profile = VisiumHDProfile()
visiumHD_profile.reset(bin_size=8)

In [10]:
visiumHD_demo = iStar_visium.Visium2HD(HDprofile=visiumHD_profile)

In [12]:
iStar_visium.set_target_VisiumHD(visiumHD_demo)

In [5]:
istarHD_dir = Path("test/istar2HD")
iStar_visium.save_inpout(istarHD_dir)

In [6]:
iStar_visium.load_output(istarHD_dir)
iStar_visium.to_VisiumHD(visiumHD_demo, istarHD_dir/"VisiumHD_result")